In [21]:
import numpy as np
np.random.seed(42)

In [20]:
import tensorflow as tf
import imp, pdb
fxns = imp.load_source('fxns', '../code/fxns.py')
re

[autoreload of fxns failed: Traceback (most recent call last):
  File "/Users/yakir/anaconda3/envs/py27/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: No module named fxns
]


In [19]:
# constants
data_path = '../data/examples'
out_path = '../output/examples'
train_out_path = '%s/train' % out_path
valid_out_path = '%s/validate' % out_path
test_out_path = '%s/test' % out_path
test_chrs = [1,2]
valid_chrs = [3,4]
blocksize = 1000
num_blocks = 4
create_data = True
text = False
binary = True
batch_size = 100
capacity = 2000
min_after_dequeue = 1000
learning_rate = 0.01

[autoreload of fxns failed: Traceback (most recent call last):
  File "/Users/yakir/anaconda3/envs/py27/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: No module named fxns
]


In [18]:
# create data
if create_data:
    fxns.create_shards(data_path,
            train_out_path, valid_out_path, test_out_path,
            test_chrs, valid_chrs,
            blocksize=blocksize,
            text=text, binary=binary, num_blocks=num_blocks)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56


KeyboardInterrupt: 

In [11]:
# get data
(seq, label), info = fxns.get_seq_and_label(train_out_path)
seq_batch, label_batch = tf.train.shuffle_batch([seq, label], batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)

In [13]:
# print data
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    _seq_batch, _label_batch = sess.run([seq_batch, label_batch])
    print _seq_batch, _label_batch
    print _seq_batch.shape, _label_batch.shape
    coord.request_stop()
    coord.join(threads)

[[ 0.  1.  0. ...,  0.  1.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 1.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  1. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]] [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
(100, 4000) (100, 24)


In [14]:
# define model
loss = fxns.get_logreg_model(seq_batch, label_batch, info['seq_len'], info['label_len'])
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

In [16]:
# train

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    sess.run(tf.global_variables_initializer())
    
    i = 0
    while True:
        _, _loss = sess.run([optimizer, loss])
        i += 1
        if i % 100 == 0:
            print i, _loss
    
    coord.request_stop()
    coord.join(threads)

100 0.114274
200 0.0548806
300 0.0357833
400 0.025815
500 0.0207835
600 0.0172614
700 0.0144076
800 0.0124637
900 0.0112763
1000 0.00998978
1100 0.00892604
1200 0.00828789
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Run call was cancelled


KeyboardInterrupt: 